# Compression, Prediction, Generation: Text Entropy


### Introduction

In this project we are interested in compressing and generating texts written in natural languages.
Given a text of length $n$, a sequence of symbols is just a vector $(x_1, . . . , x_n)$ where each $x_i$ is a symbol i.e. $x_i = a, b, c, \dots$. We can define the alphabet of possible symbols as $\mathcal{A} = \{a_1,a_2,\dots,a_M\}$ then each $x_i$ can have $M$ values.

In order to model the sequence of symbols we need a joint probability distribution for each symbol in the sequence, namely $p(X_1 = x_1, X_2 = x_2, \dots , X_n = x_n)$. If our alphabet had $M$ symbols, for modelling a sequence of length $n$ we would need $M^n$ probabilities. Thus some assumptions are required in order to reduce this dimensionality. In this case we will use two different models for $p$, the IID and the Markov Chain model.

### IID Model
The IID model assumes:

$$ p(X_1 = x_1, X_2 = x_2, \dots , X_n = x_n) = \prod_{i=1}^n p(X_i = x_i)$$

i.e. that the symbols in a sequence are independent and identically distributed. With this model we need only $M$ probabilities, one for each symbol. One can generalize and use symbols not of a single character but of multiples ones. For example using 3 characters per symbol, the symbols would be of the form $aaa,aab,...,zzz$. When using $k$ characters per symbols in an alphabet of $M$ characters, the needed probabilities would be $M^k$.


### Markov Chain Model

The Markov Chain model assume a limited range of dependence of the symbols. Indeed for an order $k$ Markov Chain:


$$p(X_i | X_{i-1},X_{i-2},\dots,X_1) = p(X_i | X_{i-1},X_{i-2},\dots,X_{i-k})$$


The meaning of the above structure is that the $i$-th symbol in the sequence depends only on the previous $k$ symbols. We add the time *invariant assumption*, meaning that the conditional probabilities do not depend on the time index $i$ i.e. $p(X_i | X_{i-1},X_{i-2},\dots,X_{i-k}) = p(X_{k+1} | X_{k},X_{k-1},\dots,X_{1})$. The most common and widely used Markov Chain is the Markov Chain of order 1:

$$p(X_i | X_{i-1},X_{i-2},\dots,X_1) = p(X_i | X_{i-1})$$

In this case the conditional probability $p(X_i|X_{i−1})$ can be expressed using $M^2$
numbers. Usually this is referred to as the *transition matrix*. Given an alphabet $\mathcal{A} = \{a_1,a_2,\dots,a_M\}$ the transition matrix can be written as: 

$$ \mathbb{M}_{kl} = p(X_i = a_k| X_{i-1} = a_l) $$

### Entropy and Cross-Entropy


- For the IID model of order 1 the entropy computation is straightforward: 
$$ H_{IID} = -\sum_{i=1}^M p(a_i) log p(a_i)$$ 
and consequently, starting from two distributions $p,q$ fitted on two different texts, the cross-entropy:
$$ CE_{IID} = -\sum_{i=1}^M p(a_i) log q(a_i)$$


- For the MC model of order 1 the entropy is defined as follows: 
$$ H_{MC} = - \sum_{kl} \pi(a_k) p(X_i = a_k| X_{i-1} = a_l) log \left(p(X_i = a_k| X_{i-1} = a_l)\right)= - \sum_{kl} \pi_k\mathbb{M}_{kl} log \mathbb{M}_{kl}$$
where $\pi$ is the stationary distribution of the Markov Chain i.e. $\pi_k = \mathbb{M}_{kl} \pi_l$. The code to compute the stationary distribution is already given.
The cross-entropy:
$$ CE_{IID} = - \sum_{kl} \pi_k\mathbb{M}_{kl} log \mathbb{M'}_{kl}$$
with $\mathbb{M}$ and $\mathbb{M'}$ are fitted on two different texts.


### Theoretical Questions: 

1) Interpret the time invariant assumption associated to our Markov chains in the contex of text generation.

This assumtion is what simplifies the Markov model, byt making the probability of transitioning from one state to another independent from when it happens, and only dependent on the state(s) itself. This allows for the use of one transition matrix regardless of the position in the text making the Markov Model much simpler. Including position in the text would exponentially raise the number of parameters and it is a hard problem to model. 

2) How can we rewrite a Markov chain of higher order as a Markov chain of order 1?

Easy, if we're dealing with an order n markov chain model, to make it order 1, we change our state space into tuples of n consecutive states in the original space. Meaning all our possible states will be all possible combinations of n states. Moving from one state to another would happen by dropping the first (oldest) state in the tuple, and adding on the new one. This would create the new tuple. 

3) Given a probability distribution over symbols, how to use it for generating sentences?

We start by randomly sampling a symbol, or maybe it is given to us. 

We then use the proba dist to randomly sample the next symbol taking into account any conditions such as the dependancies on the previous states of the Markov Model. 

Append it to our text. 

Continue until we reach the maxlength or a stopping symbol is generated (stopping conditions differ). 



### Practical questions

In order to construct our IID and Markov Chain models we need some text. Our source will be a set of classical novels available at: https://www.lri.fr/~gcharpia/informationtheory/TP2_texts.zip

We will use the symbols in each text to learn the probabilities of each model. The alphabet we suggest for the characters to use is string.printable which is made of $\sim 100$ characters. (see below)

For both models, perform the following steps:

1) For different orders of dependencies, train the model on a novel and compute the associated entropy. What do you observe as the order increases? Explain your observations.

2) Use the other novels as test sets and compute the cross-entropy for each model trained previously. How to handle symbols (or sequences of symbols) not seen in the training set?

3) For each order of dependencies, compare the cross-entropy with the entropy. Explain and interpret the differences.

4) Choose the order of dependencies with the lowest cross-entropy and generate some sentences.

5) Train one model per novel and use the KL divergence in order to cluster the novels.


<b>Hints</b> : 

- In the MC case limit yourself to order $2$ (the computation can become quite expensive). If you have $ M \sim 100$ characters, for order $1$ you will need a $\sim 100 \times 100$ matrix, for order $2$ a $\sim 10^4 \times 10^4$ matrix.

- For the second order MC model you need to compute: $p(X_{i+1},X_{i}|X_{i},X_{i-1})$

- It is possible to implement efficiently the two models with dictionaries inPython.  For the IID model, a key of the dictionary is simply a symbol and the value is the number of occurrences of the symbol in the text. For a Markov chain, a key of the dictionary is also a symbol, but the value is a vector that contains the number of occurrences of each character of the alphabet.  Notice that a symbol may consist of one or several characters. Note also that there is no need to explicitly consider all possible symbols; the ones that are observed in the training set are sufficient.

- A low probability can be assigned to symbols not observed in the training-set.

#### Computing stationary distribution 

Here we provide you two version of the function to compute the stationary distirbution of a markov chain and show a small example

In [1]:
import os
if("TP2_texts.zip" not in os.listdir(".")):
    !wget https://www.lri.fr/~gcharpia/informationtheory/TP2_texts.zip

In [2]:
#direct way to find pi (can be slow)
import  numpy  as np

def Compute_stationary_distribution(P_kl):
    ## P_kl must be the transition matrix from state l to state k!
    evals , evecs = np.linalg.eig(P_kl)   
    evec1 = evecs[:,np.isclose(evals , 1)]
    evec1 = evec1 [:,0]
    pi = evec1 / evec1.sum()
    pi = pi.real #stationary  probability
    
    return pi 

#iteative way (should be faster)
def Compute_stationary_distribution_it(P_kl, n_it):
    pi = np.random.uniform(size=P_kl.shape[0]) #initial state, can be a random one!
    pi /= pi.sum()
    #print(pi,pi.sum())
    for t in range(n_it):   
        pi = np.matmul(P_kl,pi)
    
    return pi

In [3]:
##simple example of computation of stationary distribution 

n_it = 1000                                     ##remind to check that n_it is enough to reach convergence
P_kl = np.array([[0.7,0.5],[0.3,0.5]])
Compute_stationary_distribution_it(P_kl,n_it)

array([0.625, 0.375])

#### Defining the Alphabet

Example of uploading a text and filtering out characters which are not in the chosen alphabet

In [39]:
import  string

def import_text(file_name):
    lines = []
    with  open(file_name , encoding='UTF8') as f:
        lines = f.readlines ()
        text = '\n'.join(lines)
        printable = set(string.printable)
        text = ''.join(filter(lambda x: x in printable , text))     
    return text

text = import_text('./texts/Alighieri.txt')

#### IID - MODEL

In [40]:
import numpy as np
import math
from collections import Counter 

class IIDModel:
    def __init__(self, order=1):
        self.order = order
        self.probabilities = {}  # Dictionary to store symbol probabilities
    
    def process(self, text):
        """Process the text to calculate symbol probabilities."""
        if self.order == 1:
            total_symbols = len(text)
            symbol_counts = Counter(text)  # Counts each symbol's occurrences in text
            # probability of each symbol
            self.probabilities = {symbol: count / total_symbols for symbol, count in symbol_counts.items()}
        else:
            # For higher order, group symbols
            grouped_symbols = [text[i:i+self.order] for i in range(0, len(text) - self.order + 1)]
            total_groups = len(grouped_symbols)
            symbol_counts = Counter(grouped_symbols)
            self.probabilities = {symbol: count / total_groups for symbol, count in symbol_counts.items()}
    
    def getEntropy(self):
        """Calculate and return the entropy of the model."""
        entropy = -sum(p * math.log(p, 2) for p in self.probabilities.values())
        return entropy
    
    def getCrossEntropy(self, text):
        """Calculate the cross-entropy between the model and a given text, handling unseen symbols."""
        if self.order == 1:
            symbols = text
        else:
            symbols = [text[i:i+self.order] for i in range(len(text) - self.order + 1)]
        
        unseen_prob = 1e-6  # Small probability for unseen symbols as suggested in tricks
        total_symbols = len(symbols)
        
        cross_entropy = -sum(math.log(self.probabilities.get(symbol, unseen_prob), 2) for symbol in symbols) / total_symbols
        return cross_entropy
    
    def generate(self, length):
        """Generate a text of a given length based on the model."""
        symbols, probabilities = zip(*self.probabilities.items())
        generated_text = ''.join(np.random.choice(symbols, p=probabilities) for _ in range(length))
        return generated_text


In [112]:
import math

def KL_divergence(dist1, dist2, epsilon=1e-06):
    kl_div = 0.0
    for event in dist1:
        p = dist1[event]
        q = dist2.get(event, epsilon)
        
        if p > 0 and q > 0:
            kl_div += p * math.log(p / q)
            
    return kl_div

def load_text(filepath):
    """Utility function to load text from a file."""
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.read()


## 1.
As the order increases, we notice an increase in the entropy meaning an increase in uncertainty which at first sounds controversial. But if we think about it, increasing the order means exponentially increasing the number of possible states, while assuming an IID distribution. This has 2 main problems: 

1. A system with more states is automatically more uncertain.
2. With a high number of states, without a huge corpus to clearly determine preferences, the model tends towards uniformity making entropy at its highest possible value. 

This clearly shows the mediocrity of the assumtions that the IID model makes. 

In [121]:
novel_text = load_text('./texts/Dostoevsky.txt')
test_novels = [
    load_text('./texts/Alighieri.txt'),
    load_text('./texts/Goethe.txt'),
    load_text('./texts/Hamlet.txt')]

orders = [1, 2, 3]  # Orders to test
entropies = []

for order in orders:
    model = IIDModel(order=order)
    model.process(novel_text)
    entropy = model.getEntropy()
    entropies.append((order, entropy))
    print(f"Order: {order}, Entropy: {entropy:.4f}")


Order: 1, Entropy: 4.5090
Order: 2, Entropy: 8.0591
Order: 3, Entropy: 10.8014


## 2&3.

Entropy is always smaller than cross-entropy, meaning whenever we add new information to our model, uncertainty increases. The degree or how much bigger it is, can differ depending on the language differences, writing styles ...

Note that Alligieri book has always the biggest cross-entropy. 

In [124]:
print(f'{"Order":<10} {"Test Novel":<15} {"Cross-Entropy":<15}')

for order in orders:
    model = IIDModel(order=order)
    model.process(novel_text)
    print(f"\n\tEntropy: {model.getEntropy():.4f}\n")
    for i, test_text in enumerate(test_novels, start=1):
        cross_entropy = model.getCrossEntropy(test_text)
        print(f'{order:<10} {i:<15} {cross_entropy:.4f}')

Order      Test Novel      Cross-Entropy  

	Entropy: 4.5090

1          1               4.8576
1          2               5.1693
1          3               5.0138

	Entropy: 8.0591

2          1               9.7857
2          2               10.1645
2          3               9.6282

	Entropy: 10.8014

3          1               14.3029
3          2               14.6316
3          3               14.2151


## 4. 
**Of course we were not expecting to see any meaningful sentences being generated.**

In [104]:
order = 1  
model = IIDModel(order=order)
model.process(novel_text)

for i in range(5):
    sentence = model.generate(100)  
    print(f'Sentence {i+1}: {sentence}')

Sentence 1: eolioeroy ieorel.alhw  ntca ita n uohi   oestb nhon  eiar nhehtih ot eoaam rsun?taeha eeewo
ay-abrsm
Sentence 2: srLrtn  nis  agdm i dnem  ndorfa d  
l p
 " ldeeeg wemmatieuroeo  e
aw uun"l"  gyer ecnh,fhne  g ser
Sentence 3: nnRRdlpeimd
lsuiar ! !faeslveuo eh,Slhoaugarl a 

n ewugre ssye kiml fnad"sveun seeh
hioebtkehs,tkco
Sentence 4: ib" io  gathotvshee
tws cdlikernesnK  sbaaoclhtfrv   ict  lboehi ntsa rfrm, aimrh wrhreh ncmo ele No
Sentence 5:   prr levni  n-r
 aooeehh m eaiIiskly.drrtmrfah,leb r dtoa.cls oaln  henihhsehhud ?y saRee rocv' ehf


## 5.
Here we note a big divergence between Alighieri and all the other books that lead it to be classified in its own cluster while all the other belonging to the same one.

PS: We used a hirearchical clustering approach (Agglomerative clustering) that starts with each datapoint as its own cluster and bottom-up merges datapoints that are similar to each other into the same clusters. 

In [123]:
from sklearn.cluster import AgglomerativeClustering
novels = [novel_text] + test_novels
models = []
for novel in novels:
    model = IIDModel(order=1)  #Here taking order 1 
    model.process(novel)
    models.append(model)

# Calculating the KL divergence between each pair of novels
kl_matrix = []
for i in range(len(models)):
    kl_row = []
    for j in range(len(models)):
        kl_div = KL_divergence(models[i].probabilities, models[j].probabilities)
        kl_row.append(kl_div)
    kl_matrix.append(kl_row)

clustering = AgglomerativeClustering(n_clusters=2, linkage='average')
labels = clustering.fit_predict(kl_matrix)

for i, label in enumerate(labels):
    print(f'Novel {i+1} is in cluster {label+1}')

Novel 1 is in cluster 1
Novel 2 is in cluster 2
Novel 3 is in cluster 1
Novel 4 is in cluster 1


#### MARKOV CHAIN - MODEL

In [109]:
import numpy as np
import random
from collections import defaultdict

class MarkovModel:
    def __init__(self, order=1, states=None):
        self.order = order
        self.model = defaultdict(lambda: defaultdict(int))
        if states:
            self.states = sorted(states)  
        else:
            self.states = None
        self.P_kl = None  # Transition matrix

    def process(self, text):
        if not self.states:
            self.states = sorted(set(text))  # Determine states from text if not predefined
        self.vocabulary_size = len(self.states)
        state_index = {state: i for i, state in enumerate(self.states)}

        self.P_kl = np.zeros((self.vocabulary_size, self.vocabulary_size))

        for i in range(len(text) - self.order):
            current_state = state_index.get(text[i])
            next_state = state_index.get(text[i + self.order])
            if current_state is not None and next_state is not None:
                self.P_kl[next_state, current_state] += 1
        
        self.P_kl += 1e-6  # Add a small value to avoid zero probabilities
        column_sums = self.P_kl.sum(axis=0)
        self.P_kl = self.P_kl / column_sums

    def compute_stationary_distribution(self):
        pi = np.random.uniform(size=self.vocabulary_size)
        pi /= pi.sum()
        for _ in range(1000):
            pi = np.dot(self.P_kl, pi)
        return pi

    def getEntropy(self):
        pi = self.compute_stationary_distribution()
        entropy = -np.sum(pi * np.sum(self.P_kl * np.log(self.P_kl, where=self.P_kl > 0), axis=0))
        return entropy

    def getCrossEntropy(self, other_model):
        pi = self.compute_stationary_distribution()
        cross_entropy = -np.sum(pi * np.sum(self.P_kl * np.log(other_model.P_kl, where=other_model.P_kl > 0), axis=0))
        return cross_entropy

    def generate(self, length=100):
        if self.order > len(self.states):
            raise ValueError("Order is greater than the number of unique states. Cannot generate sequence.")
        
        # starting with a random state from the existing states
        current_state_index = random.randint(0, self.vocabulary_size - 1)
        generated_text = [self.states[current_state_index]]

        for _ in range(1, length):
            current_state_probs = self.P_kl[:, current_state_index]
            if not np.any(current_state_probs > 0):
                break  # No valid next state, we break the generation loop
            current_state_index = np.random.choice(self.vocabulary_size, p=current_state_probs)
            generated_text.append(self.states[current_state_index])

        return ''.join(generated_text)
    
    def get_probabilities(self):
        return self.P_kl

## 1.

Notable decrease in entropy from the IID model. Still a slight increase from order 1 to 2. We attribute it here to the small size of the datasets we are dealing with that cannot account for the increasing size of the state space of order 2 models.

In [107]:
entropies = []

for order in [1,2]:
    model = MarkovModel(order=order)
    model.process(novel_text)
    entropy = model.getEntropy()
    entropies.append((order, entropy))
    print(f"Order: {order}, Entropy: {entropy:.4f}")

Order: 1, Entropy: 2.4608
Order: 2, Entropy: 2.8201


## 2&3.

Similar remarks to the IID case here. 

In [117]:
all_possible_characters = sorted(set(novel_text + ''.join(test_novels)))

# Initializing models using the comprehensive set of states to not encounter a dimension mismatch when calculatibg KL divergence
for order in [1, 2]:
    base_model = MarkovModel(order=order, states=all_possible_characters)
    base_model.process(novel_text)
    
    
    print(f'{"Order":<10} {"Test Novel":<15} {"Cross-Entropy":<15}')
    print(f"\n\tEntropy: {base_model.getEntropy():.4f}\n")
    for i, test_text in enumerate(test_novels, start=1):
        test_model = MarkovModel(order=order, states=all_possible_characters)
        test_model.process(test_text)
        cross_entropy = base_model.getCrossEntropy(test_model)
        print(f'{order:<10} {i:<15} {cross_entropy:.4f}')

Order      Test Novel      Cross-Entropy  

	Entropy: 2.4608

1          1               6.1064
1          2               4.3126
1          3               4.0031
Order      Test Novel      Cross-Entropy  

	Entropy: 2.8201

2          1               5.1007
2          2               3.9350
2          3               3.8570


## 4.

We do not see any meaningful text, Dostoevsky was chosen on purpose so we would recognize english words, but an order 2 Markov Chain model is still not complex enough to generate meaningful text.

In [118]:
order = 1
model = MarkovModel(order=order)
model.process(novel_text)

for i in range(5):
    sentence = model.generate(30)  
    print(f'Sentence {i+1}: {sentence}')

Sentence 1: Fovnonjean uikorespanolt ficru
Sentence 2: Svanore thit ce heceand ex we 
Sentence 3: Led... wexa ay hole ov Ang he.
Sentence 4: bu bly, thonest sha wadiker be
Sentence 5: An ranin Wandivemy edd
s t

"G


## 5. The clustering we had was the same as the IID case, confirming our previous claim.

In [119]:
def matrix_to_prob_dict(matrix, states):
    prob_dict = {}
    for i in range(len(states)):
        for j in range(len(states)):
            if matrix[i, j] > 0:
                prob_dict[(states[j], states[i])] = matrix[i, j]
    return prob_dict

In [125]:
novels = [novel_text] + test_novels  
models = []

for novel in novels:
    mm = MarkovModel(order=2) 
    mm.process(novel)
    models.append(mm)

kl_matrix = []
for i in range(len(models)):
    kl_row = []
    prob1 = matrix_to_prob_dict(models[i].get_probabilities(), models[i].states)
    for j in range(len(models)):
        prob2 = matrix_to_prob_dict(models[j].get_probabilities(), models[j].states)
        kl_div = KL_divergence(prob1, prob2)
        kl_row.append(kl_div)
    kl_matrix.append(kl_row)

clustering = AgglomerativeClustering(n_clusters=2, linkage='average')
labels = clustering.fit_predict(kl_matrix)
for i, label in enumerate(labels):
    print(f'Novel {i+1} is in cluster {label+1}')


Novel 1 is in cluster 1
Novel 2 is in cluster 2
Novel 3 is in cluster 1
Novel 4 is in cluster 1
